In [ ]:
import pandas as pd
from datetime import time
from ipyfilechooser import FileChooser
from IPython.display import display
import os


# 🎛️ Interface de sélection de fichier
fc_input = FileChooser("C:/Users/AudreyHay/Documents/Carla/FED/", select_default=True,show_only_dirs=False, title="<b>Sélectionnez le fichier CSV</b>")
display(fc_input)

# 📂 Fonction appelée après sélection du fichier
def update_input_file(chooser):
    global input_file_path
    input_file_path = chooser.selected
    print(f"\n📂 Fichier sélectionné : {input_file_path}")



In [ ]:
df = pd.read_csv(input_file_path)
df.columns = df.columns.str.replace(r'\+AF8-', '_', regex=True)
df

In [ ]:
# 📌 Colonnes à supprimer
cols_to_drop = [
    'Library_Version', 'Session_type', 'Device_Number',
    'Battery_Voltage', 'Motor_Turns', 'FR'
]   
# 🧹 Supprimer les colonnes indésirables
df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)
df

In [ ]:

# 🕒 Convertir la colonne datetime
df['Datetime'] = pd.to_datetime(df['MM:DD:YYYY hh:mm:ss'], errors='coerce')

# 🗓️ Extraire Date et Hour
df['Date'] = df['Datetime'].dt.date
df['Hour'] = df['Datetime'].dt.time

#df.rename(columns={'Active_Poke': 'ActivePoke'}, inplace=True)

# ☀️ Déterminer la période Day/Night
def get_period(hour):
    if pd.isnull(hour):
        return None
    if time(7, 0, 1) <= hour <= time(19, 0, 0):
        return "Day"
    else:
        return "Night"

df['Period'] = df['Hour'].apply(get_period)

# 💾 Enregistrer dans le même dossier avec _modifié
folder = os.path.dirname(input_file_path)
base = os.path.splitext(os.path.basename(input_file_path))[0]
output_file = os.path.join(folder, f"{base}_Periodic.csv")

df

In [ ]:
df.to_csv(output_file, index=False)

# ✅ Confirmation + aperçu
print("\n✅ Le fichier a bien été enregistré !")
print(f"📍 Chemin : {output_file}")
print("\n🔍 Aperçu des premières lignes du fichier modifié :")
display(df.head())


In [ ]:
# Afficher les événements uniques pour vérification
print("Types d'événements dans les données :")
print(df['Event'].unique())

# Assurer format cohérent
df['Datetime'] = pd.to_datetime(df['MM:DD:YYYY hh:mm:ss'], errors='coerce')
df['Date'] = pd.to_datetime(df['Date'], errors='coerce').dt.date
df['Hour'] = pd.to_datetime(df['Hour'], format='%H:%M:%S', errors='coerce').dt.time

# Trier les données par date et heure
df = df.sort_values(['Date', 'Hour']).reset_index(drop=True)

# Créer un identifiant de bloc pour les périodes consécutives
# Cela va identifier chaque changement de période (Day->Night ou Night->Day)
df['Period_Block'] = (df['Period'] != df['Period'].shift(1)).cumsum()

# Définir les catégories d'événements
specific_events = [
    'LeftDuringDispense',
    'RightDuringDispense',
    'LeftWithPellet',
    'RightWithPellet',
    'RightinTimeOut',
    'LeftinTimeOut'
]

results = []

# Regrouper par bloc de période (périodes consécutives)
for (period_block), group in df.groupby('Period_Block'):
    # Le period_type est le même pour tout le groupe (Day ou Night)
    period_type = group['Period'].iloc[0]
    
    left_pokes = group[group['Event'] == 'Left'].shape[0]
    right_pokes = group[group['Event'] == 'Right'].shape[0]
    total_pokes = left_pokes + right_pokes
    correct = (group['Event'] == group['Active_Poke']).sum()
    
    # Comptage des pellets
    pellets = group[group['Event'] == 'Pellet'].shape[0]
    
    # First poke après Pellet
    pellet_indices = group[group['Event'] == 'Pellet'].index
    first_pokes = []
    for idx in pellet_indices:
        if idx + 1 in group.index:
            first_pokes.append(group.loc[idx + 1])
    first_pokes_df = pd.DataFrame(first_pokes)
    first_poke_total = len(first_pokes)
    first_poke_correct = (first_pokes_df['Event'] == first_pokes_df['Active_Poke']).sum() if not first_pokes_df.empty else 0
    
    # Calcul des pourcentages
    pct_first_poke_correct = round((first_poke_correct / first_poke_total) * 100, 2) if first_poke_total > 0 else None
    pct_correct = round((correct / total_pokes) * 100, 2) if total_pokes > 0 else None
    
    # Specific events
    spec_events = group[group['Event'].isin(specific_events)].shape[0]
    
    # Dates de début et fin du bloc
    start_date = group['Date'].min()
    end_date = group['Date'].max()
    
    results.append({
        'Period Block': period_block,
        'Period Type': period_type,
        'Start Date': start_date,
        'End Date': end_date,
        'Pellet': pellets,
        'Pokes': total_pokes,
        'Left Pokes': left_pokes,
        'Right Pokes': right_pokes,
        'Correct': correct,
        '% Correct': pct_correct,
        'First Poke': first_poke_total,
        'First Poke Correct': first_poke_correct,
        '% First Poke Correct': pct_first_poke_correct,
        'Specific Events': spec_events,
    })

# Résumé
summary_df = pd.DataFrame(results)
summary_df

In [ ]:
# Sauvegarde CSV
base = os.path.splitext(os.path.basename(output_file))[0]
output_summary = os.path.join(folder, f"{base}_global.csv")
summary_df.to_csv(output_summary, index=False)

print(f"\n✅ Fichier résumé sauvegardé avec succès !")
print(f"📍 Chemin : {output_summary}")
display(summary_df)